Download the csv file from the azure blobs with in the date range and insert into data base

In [27]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
import psycopg2
import datetime
from io import StringIO

In [28]:
account_name = "daywisebilling"
STORAGEACCOUNTURL = f"https://{account_name}.blob.core.windows.net"
STORAGEACCOUNTKEY = "jaCG7aC1Ilp7vpe9xDko8eMwqMTMiFNyY+ArzFTTkoutRQGkAt7KqkeB1HOX4d7cWyKKatRLLIxI+AStcecHHA=="
CONTAINERNAME = "day-wise-billing"
BLOBNAME_PREFIX = "dev/DailyBilling/"
# Define the storage account name and access key
account_name = "daywisebilling"
account_key = "jaCG7aC1Ilp7vpe9xDko8eMwqMTMiFNyY+ArzFTTkoutRQGkAt7KqkeB1HOX4d7cWyKKatRLLIxI+AStcecHHA=="

In [31]:
def extract_date_from_blob_name(blob_name):
    # Extract the date from the blob name in the format "20230101-20230131"
    date_str = blob_name.split("/")[-2]
    return date_str

# Create a BlobServiceClient using the account URL and credentials
blob_service_client = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=account_key)

# Get a reference to the desired container
container_client = blob_service_client.get_container_client(CONTAINERNAME)

# Get a list of all blobs in the container
blobs = container_client.list_blobs()

start_date = "20230101"
end_date = "20230131"

# Create an empty list to store the matching blobs
matching_blobs = []

# Iterate through the list of blobs and find the ones that match the date range
for blob in blobs:
    date_str = extract_date_from_blob_name(blob.name)
    if start_date <= date_str <= end_date and ".csv" in blob.name:
        # Add the matching blob to the list
        matching_blobs.append(blob)

# Find the latest updated blob from the matching list
latest_blob = max(matching_blobs, key=lambda x: x.last_modified)

# Get a reference to the latest updated blob
blob_client = container_client.get_blob_client(latest_blob.name)

print(f"Found latest updated blob: {latest_blob.name}")

Found latest updated blob: dev/DailyBilling/20230101-20230131/DailyBilling_8b5b095a-5afe-4810-bbb8-04843145a473.csv


In [20]:
def get_columns():
    connection = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
    try:
        cursor = connection.cursor()
        Query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'azure_data';"
        cursor.execute(Query)
        records = cursor.fetchall()
        # connection.commit()
        columns = []
        for i in records:
            columns.append(i[0])
        # columns=columns
        return columns
    except:
        print("check whether the database exits or not")

In [32]:
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
# columns = get_columns()
cursor = conn.cursor()
stream = container_client.get_blob_client(latest_blob).download_blob()
df = pd.read_csv(StringIO(stream.content_as_text()))
df.columns=map(str.lower,df.columns)
# print(df)
buffer = StringIO()
columns = get_columns()
df = df[columns]

df.to_csv(buffer, index = False ,sep="\t",header = False)
buffer.seek(0)

cursor.copy_from(buffer, "azure_data", sep="\t", null="")
print("Data is Inserted")
# print(stream)
conn.commit()

cursor.close()
conn.close()





Data is Inserted


Checks the condition if the data is already inserted or not

In [ ]:
# Connect to the database
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
cursor = conn.cursor()

# Get the latest CSV file
stream = container_client.get_blob_client(latest_blob).download_blob()
df = pd.read_csv(StringIO(stream.content_as_text()))                                                  #Checks the condition if the date ia already inserted or not
df.columns=map(str.lower,df.columns)

# Get the dates that are already present in the database
query = "SELECT usagedatetime FROM azure_data"
cursor.execute(query)
existing_dates = [row[0] for row in cursor.fetchall()]

# Filter the data in the CSV file to only include the dates that are not present in the database
df = df[~df['usagedatetime'].isin(existing_dates)]

# Insert the filtered data into the database
buffer = StringIO()
columns = get_columns()
df = df[columns]
df.to_csv(buffer, index = False ,sep="\t",header = False)
buffer.seek(0)
cursor.copy_from(buffer, "azure_data", sep="\t", null="")
print("Data is Inserted")
conn.commit()

# Close the database connection
cursor.close()
conn.close()

Download te latest updated csv file from every month

In [22]:
import calendar,os

# Create a BlobServiceClient using the account URL and credentials
blob_service_client = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=account_key)

# Get a reference to the desired container
container_client = blob_service_client.get_container_client(CONTAINERNAME)

local_directory = r"C:\\Users\\p.likith varma\\Desktop\\billing_data"

def extract_date_from_blob_path(blob_path):
    date_str = blob_path.split("/")[-2]
    start_date, end_date = date_str.split("-")
    return start_date, end_date
                                                                         # Download the latest updated file from every month!!!!!!!!!!!!!!!
blobs = container_client.list_blobs()

# Keep track of the latest updated file for each month
latest_files = {}

for blob in blobs:
    start_date_blob, end_date_blob = extract_date_from_blob_path(blob.name)
    start_month = start_date_blob[:6]
    end_month = end_date_blob[:6]

    if start_month not in latest_files:
        latest_files[start_month] = None

    # Check if the blob is a CSV file and if it's the latest for its month
    if ".csv" in blob.name and (not latest_files[start_month] or blob.last_modified > latest_files[start_month].last_modified):
        latest_files[start_month] = blob

for start_month, blob in latest_files.items():
    start_date = start_month + "01"
    end_date = start_month + str(calendar.monthrange(int(start_month[:4]), int(start_month[4:]))[1])
    print(start_date)

    # Get a reference to the latest updated CSV file for this month
    blob_client = container_client.get_blob_client(blob.name)
    print(start_date)
    # Download the contents of the file
    local_file_path = os.path.join(local_directory, os.path.basename(blob.name))
    with open(local_file_path, "wb") as f:
        f.write(blob_client.download_blob().readall())

    # Do something with the file contents...


20230101
20230101
20230201
20230201


Download latest file of everymonth and insert into the database

In [ ]:
import calendar,os

# Create a BlobServiceClient using the account URL and credentials
blob_service_client = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=account_key)

# Get a reference to the desired container
container_client = blob_service_client.get_container_client(CONTAINERNAME)

local_directory = r"C:\\Users\\p.likith varma\\Desktop\\billing_data"

def extract_date_from_blob_path(blob_path):
    date_str = blob_path.split("/")[-2]
    start_date, end_date = date_str.split("-")
    return start_date, end_date
                                                                         # Download the latest updated file from every month!!!!!!!!!!!!!!!
blobs = container_client.list_blobs()

# Keep track of the latest updated file for each month
latest_files = {}

for blob in blobs:
    start_date_blob, end_date_blob = extract_date_from_blob_path(blob.name)
    start_month = start_date_blob[:6]
    end_month = end_date_blob[:6]

    if start_month not in latest_files:
        latest_files[start_month] = None

    # Check if the blob is a CSV file and if it's the latest for its month
    if ".csv" in blob.name and (not latest_files[start_month] or blob.last_modified > latest_files[start_month].last_modified):
        latest_files[start_month] = blob

for start_month, blob in latest_files.items():
    start_date = start_month + "01"
    end_date = start_month + str(calendar.monthrange(int(start_month[:4]), int(start_month[4:]))[1])
    print(start_date)

    # Get a reference to the latest updated CSV file for this month
    blob_client = container_client.get_blob_client(blob.name)
    
    print(start_date)
    # Download the contents of the file
    # local_file_path = os.path.join(local_directory, os.path.basename(blob.name))
    # with open(local_file_path, "wb") as f:
    #     f.write(blob_client.download_blob().readall())
    
    conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
# columns = get_columns()
    cursor = conn.cursor()
    stream = container_client.get_blob_client(blob.name).download_blob()
    df = pd.read_csv(StringIO(stream.content_as_text()))
    df.columns=map(str.lower,df.columns)
    # print(df)
    buffer = StringIO()
    columns = get_columns()
    df = df[columns]

    df.to_csv(buffer, index = False ,sep="\t",header = False)
    buffer.seek(0)

    cursor.copy_from(buffer, "azure_data", sep="\t", null="")
    print("Data is Inserted")
    # print(stream)
    conn.commit()

  

    # Do something with the file contents...
def get_columns():
    connection = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
    try:
        cursor = connection.cursor()
        Query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'azure_data';"
        cursor.execute(Query)
        records = cursor.fetchall()
        # connection.commit()
        columns = []
        for i in records:
            columns.append(i[0])
        # columns=columns
        return columns
    except:
        print("check whether the database exits or not")
        

cursor.close()
conn.close()  

Download latest csv file from every month and import it into database and check wheather the file is existing or not


In [34]:
import calendar,os

# Create a BlobServiceClient using the account URL and credentials
blob_service_client = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=account_key)

# Get a reference to the desired container
container_client = blob_service_client.get_container_client(CONTAINERNAME)

local_directory = r"C:\\Users\\p.likith varma\\Desktop\\billing_data"

def extract_date_from_blob_path(blob_path):
    date_str = blob_path.split("/")[-2]
    start_date, end_date = date_str.split("-")
    return start_date, end_date
                                                                         # Download the latest updated file from every month!!!!!!!!!!!!!!!
blobs = container_client.list_blobs()

# Keep track of the latest updated file for each month
latest_files = {}

for blob in blobs:
    start_date_blob, end_date_blob = extract_date_from_blob_path(blob.name)
    # print(start_date_blob)
    # print(end_date_blob)
    start_month = start_date_blob[:6]
    end_month = end_date_blob[:6]

    if start_month not in latest_files:
        latest_files[start_month] = None

    # Check if the blob is a CSV file and if it's the latest for its month
    if ".csv" in blob.name and (not latest_files[start_month] or blob.last_modified > latest_files[start_month].last_modified):
        latest_files[start_month] = blob    

for start_month, blob in latest_files.items():
    start_date = start_month + "01"
    end_date = start_month + str(calendar.monthrange(int(start_month[:4]), int(start_month[4:]))[1])
    print(start_date)
    print(end_date)
    # Get a reference to the latest updated CSV file for this month
    blob_client = container_client.get_blob_client(blob.name)
    
    # print(start_date)
    # Download the contents of the file
    # local_file_path = os.path.join(local_directory, os.path.basename(blob.name))
    # with open(local_file_path, "wb") as f:
    #     f.write(blob_client.download_blob().readall())
    
    conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Cescost#123",
    host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
    port='5432'
)
# columns = get_columns()
    cursor = conn.cursor()
    stream = container_client.get_blob_client(blob.name).download_blob()
    df = pd.read_csv(StringIO(stream.content_as_text()))
    df.columns=map(str.lower,df.columns)
    
    # Get the dates that are already present in the database
    query = "SELECT usagedatetime FROM azure_data"
    cursor.execute(query)
    existing_dates = [row[0] for row in cursor.fetchall()]

# Filter the data in the CSV file to only include the dates that are not present in the database
    df = df[~df['usagedatetime'].isin(existing_dates)]
    # print(df)
    buffer = StringIO()
    # columns = get_columns()
    # df = df[columns]

    df.to_csv(buffer, index = False ,sep="\t",header = False)
    buffer.seek(0)

    cursor.copy_from(buffer, "azure_data", sep="\t", null="")
    print("Data is Inserted")
    # print(stream)
    conn.commit()

  

    # Do something with the file contents...
# def get_columns():
#     connection = psycopg2.connect(
#     database="postgres",
#     user="postgres",
#     password="Cescost#123",
#     host="ces-cost-opt-01.c9girek67w2d.us-east-1.rds.amazonaws.com",
#     port='5432'
# )
#     try:
#         cursor = connection.cursor()
#         Query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'azure_data';"
#         cursor.execute(Query)
#         records = cursor.fetchall()
#         # connection.commit()
#         columns = []
#         for i in records:
#             columns.append(i[0])
#         # columns=columns
#         return columns
#     except:
#         print("check whether the database exits or not")
        

cursor.close()
conn.close()  

20230101
20230131
Data is Inserted
20230201
20230228
Data is Inserted
